In [2]:
# prepare data

data = []
with open('dialog_acts.dat', 'rb') as file:
    for line in file:
        decoded_line = line.decode('utf-8').strip().lower()
        words = decoded_line.split(' ', 1)  
        data.append(words)

In [3]:
# create x and y(target) datasets
X = [item[1] for item in data] 
y = [item[0] for item in data] 

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85, random_state=42)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
## Baseline systems

## Baseline 'inform'

def create_y_pred_inform(data):
    y_pred = []
    for string in data:
        y_pred.append('inform')
    return y_pred


## Baseline 'manual rules'
rules = {
    'ack': ['okay', 'kay', 'um', 'good', 'thatll do'],
    'affirm': ['yes', 'right', 'correct'],
    'bye': ['goodbye', 'see', 'bye'],
    'confirm': ['is', 'there', 'right', 'confirm'],
    'deny': ['dont', 'don\'t' 'reject',],
    'hello': ['hi', 'hello', 'greetings', 'hey'],
    'inform': ['looking', 'restaurant', 'serves', 'seafood', 'preference', 'any area', 'information'],
    'negate': ['no', 'not', 'negate', 'none'],
    'null': ['cough', 'noise', 'uh', 'umm'],
    'repeat': ['repeat', 'try', 'back'],
    'reqalts': ['about', 'alternative', 'other options', 'korean food'],
    'reqmore': ['more', 'more suggestions', 'give me more'],
    'request': ['what is', 'post code', 'ask', 'information'],
    'restart': ['start over', 'restart', 'begin again'],
    'thankyou': ['thank you', 'thanks', 'much appreciated']
}

def create_y_pred_rules(data, rules):
    y_pred = []
    for string in data:
        found = False 
        for label, keywords in rules.items():
            if any(keyword in string.lower() for keyword in keywords): ## we have a problem: if more keywords are found, then it adds both. How do we tackle this?
                y_pred.append(label)
                found = True
        if not found:
            y_pred.append('inform')

    return y_pred




In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

## Out of vocabulary dummy
OOV_TOKEN = "<OOV>"

# Bag of words representation using CountVectorizer
#vectorizer = CountVectorizer()
vectorizer = CountVectorizer(tokenizer=lambda x: x.split()) ## we need to ensure we use tokens that are split, could also do it in preprocessing when we configure the data initially
vectorizer.fit(X_train)
vectorizer.vocabulary_[OOV_TOKEN] = len(vectorizer.vocabulary_) ## add the oov_token to our vocabulary i.e. the '0' integer in assignment

X_train_matrix = vectorizer.fit_transform(X_train) ## Bag of words representation  


## Now for every word that appears in X_test that does not appear in X_train, we need to replace it to the <OOV> token dummy.

def tokenize_and_replace_oov(sentences, vocabulary):
    processed_sentences = []
    for sentence in sentences:
        tokens = sentence.lower().split()
        # Replace OOV words with OOV_TOKEN
        tokens = [token if token in vocabulary else OOV_TOKEN for token in tokens]
        processed_sentences.append(' '.join(tokens))
    return processed_sentences

test_processed = tokenize_and_replace_oov(X_test, vocabulary=vectorizer.vocabulary_)
X_test_matrix = vectorizer.transform(test_processed) ## Bag of words representation

# We need to encode the labels because we need to convert categorical string labels into a numerical format for machine learning:
# what it does: it makes the string labels into numbers so the ML model can use them for computational processing, the numbers dont mean anything inherently however.

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# (Naive Bayes)
model = MultinomialNB()
model.fit(X_train_matrix, y_train_encoded)

# (Logistic Regression)
model2 = LogisticRegression(max_iter=1000)
model2.fit(X_train_matrix, y_train_encoded)

# Evaluate
y_pred_nb = model.predict(X_test_matrix)
y_pred_labels_nb = label_encoder.inverse_transform(y_pred_nb)
y_pred_lr = model2.predict(X_test_matrix)
y_pred_labels_lr = label_encoder.inverse_transform(y_pred_lr)

y_pred_labels_inform = create_y_pred_inform(X_test)
y_pred_labels_rules = create_y_pred_rules(X_test, rules)
print(len(y_pred_labels_rules))

# Print classification reports
print('Naive Bayes classification report:')
print(classification_report(y_test, y_pred_labels_nb))

print('Logistic Regression classification report:')
print(classification_report(y_test, y_pred_labels_lr))

print('Automatic Inform classification report:')
print(classification_report(y_test, y_pred_labels_inform))

#print('Rule based classification report:')
#print(classification_report(y_test, y_pred_labels_rules))





5525
Naive Bayes classification report:
             precision    recall  f1-score   support

        ack       0.00      0.00      0.00         5
     affirm       0.99      0.92      0.95       180
        bye       0.00      0.00      0.00        35
    confirm       0.82      0.64      0.72        22
       deny       0.00      0.00      0.00         6
      hello       1.00      0.29      0.44        14
     inform       0.95      0.96      0.96      1532
     negate       0.97      0.84      0.90        69
       null       0.93      0.81      0.87       232
     repeat       1.00      0.67      0.80         3
    reqalts       0.84      0.97      0.91       279
    reqmore       0.00      0.00      0.00         1
    request       0.97      0.97      0.97       972
    restart       1.00      0.50      0.67         2
   thankyou       0.93      1.00      0.96       474

avg / total       0.93      0.94      0.93      3826

Logistic Regression classification report:
             

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
deduplicated_data = {}
for sentence, label in zip(X, y):
    if sentence not in deduplicated_data:
        deduplicated_data[sentence] = label

# Extract deduplicated X and y
X_unique = list(deduplicated_data.keys())
y_unique = list(deduplicated_data.values())

X_train_u, X_test_u, y_train_u, y_test_u = train_test_split(X_unique, y_unique, train_size=0.85, random_state=42)

vectorizer_u = CountVectorizer(tokenizer=lambda x: x.split())
vectorizer_u.fit(X_train_u)
vectorizer_u.vocabulary_[OOV_TOKEN] = len(vectorizer_u.vocabulary_)
X_train_matrix_u = vectorizer_u.transform(X_train_u)
X_test_processed_u = tokenize_and_replace_oov(X_test_u, vocabulary=vectorizer_u.vocabulary_)
X_test_matrix_u = vectorizer_u.transform(X_test_processed_u)

label_encoder_u = LabelEncoder()
y_train_encoded_u = label_encoder_u.fit_transform(y_train_u)
y_test_encoded_u = label_encoder_u.transform(y_test_u)

model_nb_u = MultinomialNB()
model_nb_u.fit(X_train_matrix_u, y_train_encoded_u)

y_pred_nb_u = model_nb_u.predict(X_test_matrix_u)
y_pred_labels_nb_u = label_encoder_u.inverse_transform(y_pred_nb_u)
print(classification_report(y_test_u, y_pred_labels_nb_u))

model_lr_u = LogisticRegression(max_iter=1000)
model_lr_u.fit(X_train_matrix_u, y_train_encoded_u)

y_pred_lr_u = model_lr_u.predict(X_test_matrix_u)
y_pred_labels_lr_u = label_encoder_u.inverse_transform(y_pred_lr_u)
print(classification_report(y_test_u, y_pred_labels_lr_u))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


             precision    recall  f1-score   support

        ack       0.00      0.00      0.00         3
     affirm       1.00      0.38      0.55        24
        bye       0.00      0.00      0.00         9
    confirm       0.69      0.53      0.60        17
       deny       0.00      0.00      0.00         1
      hello       1.00      0.20      0.33         5
     inform       0.85      0.93      0.89       447
     negate       0.88      0.56      0.68        25
       null       0.44      0.09      0.15        46
     repeat       0.00      0.00      0.00         1
    reqalts       0.75      0.92      0.83        77
    request       0.82      0.92      0.86       135
    restart       0.00      0.00      0.00         1
   thankyou       0.57      0.92      0.71        13

avg / total       0.80      0.82      0.79       804



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

        ack       0.00      0.00      0.00         3
     affirm       1.00      0.83      0.91        24
        bye       1.00      0.56      0.71         9
    confirm       0.94      0.88      0.91        17
       deny       0.00      0.00      0.00         1
      hello       1.00      0.80      0.89         5
     inform       0.88      0.97      0.92       447
     negate       0.96      0.96      0.96        25
       null       0.50      0.09      0.15        46
     repeat       1.00      1.00      1.00         1
    reqalts       0.86      0.88      0.87        77
    request       0.93      0.96      0.95       135
    restart       1.00      1.00      1.00         1
   thankyou       0.92      0.92      0.92        13

avg / total       0.87      0.89      0.87       804



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
def Interactive_Prediction():
    print("enter a sentence or type 'exit' or 'quit' to end):") ## other wise it does not end
    while True:
        user_input = input().strip().lower() ## save the input
        if user_input in ['exit', 'quit']:
            break
        else:
            process_input = tokenize_and_replace_oov([user_input], vocabulary=vectorizer.vocabulary_)
            input_matrix = vectorizer.transform(process_input)
            pred = model2.predict(input_matrix)
            label = label_encoder.inverse_transform(pred)
            print('Logistic Regression predicts:' + label)




In [9]:
def expandRestaurantCSV():
    with open('restaurant_info.csv', 'r') as csvinput:
        with open('restaurant_info_1c.csv', 'w', newline='') as csvoutput:
            writer = csv.writer(csvoutput, lineterminator='\n')
            reader = csv.reader(csvinput)

            all_rows = []
            header = next(reader)  # Read the header row
            header.extend(['goodfood', 'busy', 'longstay'])  # Append new columns
            all_rows.append(header)

            for row in reader:
                row.append(random.choice([True, False]))  # Add 'goodfood' value
                row.append(random.choice([True, False]))  # Add 'busy' value
                row.append(random.choice([True, False]))  # Add 'longstay' value
                all_rows.append(row)

            writer.writerows(all_rows)

In [21]:
import csv
import Levenshtein
import random

additional_req = ""
restaurants = []
with open('restaurant_info_1c.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Clean up the keys and values
        restaurant = {
            'name': row['restaurantname'].strip().lower(),
            'area': row['area'].strip().lower(),
            'price': row['pricerange'].strip().lower(),
            'food': row['food'].strip().lower(),
            'phone': row['phone'].strip(),
            'addr': row['addr'].strip(),
            'postcode': row['postcode'].strip().lower(),
            'goodfood': row['goodfood'],
            'busy': row['busy'],
            'longstay': row['longstay']
            }
        restaurants.append(restaurant)

        

AREAS = list(set(restaurant['area'] for restaurant in restaurants))
PRICE_RANGES = list(set(restaurant['price'] for restaurant in restaurants))
FOOD_TYPES = list(set(restaurant['food'] for restaurant in restaurants))

dont_care_signal = ['any', 'dont care', 'dontcare', 'don\'t care', 'whatever']

AREAS += dont_care_signal
PRICE_RANGES += dont_care_signal
FOOD_TYPES += dont_care_signal

ADDITIONAL_REQ = ['romantic', 'touristic', 'assigned seats', 'children']

flag_dict = {
    'area': AREAS,
    'price': PRICE_RANGES,
    'food': FOOD_TYPES,
    #'additional_req': ADDITIONAL_REQ
}



def Interactive():
    next_state = 'start'
    preferences = {}
    suggested = {}
    rejected = []
    additional_req = ""
    print("Type 'exit' or 'quit' to end the conversation at any time. Let's begin.")
    print('Hello, this is a restaurant finder system. Please tell us about which direction you want to go (north, west etc.), what price range (cheap, moderate, expensive), and what type of food you would like.')
    print('Please type in a simple manner, I am not very smart and very rigorous.')
    while True:
        print("user:")
        user_input = input().strip().lower()
        if user_input in ['exit', 'quit']:
            print("Thank you for using the restaurant finder system. Goodbye!")
            break
        if user_input in ['restart']:
            print('restarting system now')
            preferences = {}
            suggested = {}
            rejected = []
        next_state, system_response, suggested = state_transition(user_input, next_state, preferences, suggested, rejected)
        print(f"System: {system_response}")

def state_transition(user_input, state, preferences, suggested, rejected):
    dialog_act = classify_dialog_act(user_input)
    print(dialog_act)
    handler = dialog_act_handlers.get(dialog_act)
    system_response = ''
    next_state, system_response, suggested = handler(state, user_input, preferences, suggested, rejected)
    return next_state, system_response, suggested

def classify_dialog_act(user_input):
    process_input = tokenize_and_replace_oov([user_input], vocabulary=vectorizer.vocabulary_)
    input_matrix = vectorizer.transform(process_input)
    pred = model2.predict(input_matrix)
    label = label_encoder.inverse_transform(pred)
    return label[0]

def recommend_restaurant(preferences, suggested, rejected):
    ## don't recommend the same restaurant when asked for another
    
    #delete key's with don't care signal
    pref_wo_any = {key: value for key, value in preferences.items() if value.lower() not in dont_care_signal} 
    
    matching_restaurants = [
        restaurant for restaurant in restaurants
        if all(restaurant.get(key) == value for key, value in pref_wo_any.items())
    ]
    
    matching_restaurants = [restaurant for restaurant in matching_restaurants if restaurant not in rejected]
    
    if not matching_restaurants:
        max_matches = 0
        best_matches = []
        for restaurant in restaurants:
            matches = sum(
                1 for key, value in pref_wo_any.items()
                if restaurant.get(key) == value
            )
            if matches > max_matches:
                max_matches = matches
                best_matches = [restaurant]
            elif matches == max_matches:
                best_matches.append(restaurant)
        
        if best_matches and max_matches > 0:
            return random.choice(best_matches), False
        if max_matches == 0:
            return random.choice(restaurants), False
        
    matching_restaurants = [restaurant for restaurant in matching_restaurants if restaurant not in rejected]
    return matching_restaurants, True

#print(recommend_restaurant({ 'area': 'north'}, [], []))

additional_requiremenst_list = ['romantic', 'touristic', 'assigned seats', 'children', 'no']
#['goodfood', 'busy', 'longstay']

def additionalRequirements (restaurants, additional_req):
    adreqres = []
    found = False
    
    for restaurant in restaurants:
        if restaurant['goodfood'] and restaurant['price'] == 'cheap':
            restaurant['touristic'] = True
        if restaurant['food'] == 'romanian':
            restaurant['touristic'] = False
        if restaurant['busy']:
            restaurant['assigned seats'] = True
            restaurant['romantic'] = False
        if restaurant['longstay']:
            restaurant['children'] = False
            restaurant['romantic'] = True
    for restaurant in restaurants:
        if restaurant[additional_req]:
            adreqres.append(restaurant)
    if adreqres:
        found = True
        return adreqres, found
    else:
        return restaurants, found
            


def handle_inform2(state, user_input, preferences, suggested, rejected):
    if state in ['start', 'get_preferences']:
        next_state = 'get_preferences'
        user_words = user_input.lower().split()
        for word in user_words:
            for key, values in flag_dict.items():
                if word in [v.lower() for v in values]:
                    preferences[key] = word
        


def handle_inform(state, user_input, preferences, suggested, rejected):
    additional_req = ""
    if state in ['start', 'get_preferences']:
        next_state = 'get_preferences'
        user_words = user_input.lower().split()
        matched = False
        for word in user_words:
            for key, values in flag_dict.items():
                if word in [v.lower() for v in values] and not key in preferences:
                    preferences[key] = word
                    matched = True
            if word in additional_requiremenst_list:
                additional_req = word
                matched = True
        if matched:
            if all(pref in preferences for pref in ['area', 'price', 'food']):

                restaurants, match = recommend_restaurant(preferences, suggested, rejected)
                
                if len(restaurants) > 1 and match and not additional_req:
                    system_response = "Do you have additional requirements? ('romantic', 'touristic', 'assigned seats', 'children', 'no')"
                    next_state = 'get_preferences'
                else:
                    next_state = 'suggest_restaurant'
                    ar_restaurants, found = additionalRequirements(restaurants, additional_req)
                    if found:
                        suggested = random.choice(ar_restaurants)
                        if match: system_response = f"Great! A restaurant that matches all your preferences is the {restaurant['name']}, it is {additional_req}."
                        else: system_response = f"We have found no restaurant that matches the entire description of your preferences. But, a close match is {restaurant['name']}, it is a {restaurant['food']} type restaurant, the price range is {restaurant['price']} and it is {restaurant['area']}."
                    else:
                        suggested = random.choice(restaurants)
                        if match: system_response = f"Great! A restaurant that matches almost all your preferences is the {restaurant['name']}, unfortunatly it's not {additional_req}."
                        else: system_response = f"We have found no restaurant that matches the entire description of your preferences. But, a close match is {restaurant['name']}, it is a {restaurant['food']} type restaurant, the price range is {restaurant['price']} and it is {restaurant['area']}."
                    
            else:
                response_prefs = ', '.join(f"{key}: {value}" for key, value in preferences.items())
                #system_response = f"Got it. You prefer {response_prefs}. Let's continue."
                #print(preferences)
                if not 'area' in preferences:
                    system_response = f"Got it. You prefer {response_prefs}. Let's continue, what area?"
                elif not 'price' in preferences:
                    system_response = f"Got it. You prefer {response_prefs}. Let's continue, what price?"
                elif not 'food' in preferences:
                    system_response = f"Got it. You prefer {response_prefs}. Let's continue, what food type?"
                ## in system_response, vraag naar wat mist. (in volgorde: area ->  price -> food)
        else:
            min_distance = float('inf')
            best_match = None
            best_key = None
            for word in user_words:
                for key, values in flag_dict.items():
                    for value in values:
                        distance = Levenshtein.distance(word, value.lower())
                        if distance < min_distance:
                            min_distance = distance
                            best_match = value
                            best_key = key
            if min_distance <= 2 and best_match and best_key:
                preferences[best_key] = best_match.lower()
                matched = True
                if all(pref in preferences for pref in ['area', 'price', 'food']):
                    next_state = 'suggest_restaurant'
                    restaurants, match = recommend_restaurant(preferences, suggested, rejected)
                    suggested = random.choice(restuarants)
                    if match:
                        system_response = f"Great! A restaurant that matches all your preferences is the {restaurant['name']}."
                    else: 
                        system_response = f"We have found no restaurant that matches the entire description of your preferences. But, a close match is {restaurant['name']}, it is a {restaurant['food']} type restaurant, the price range is {restaurant['price']} and it is {restaurant['area']}."
                else:
                    ## wat weet hij al? vraag naar wat mist, in de volgorde van het model
                    system_response_p1 = f"Got it. You prefer {best_match} {best_key}. If this isn't what you meant, type restart. Let's continue, "
                    if not 'area' in preferences:
                        system_response_p2 = "what area?"
                    elif not 'price' in preferences:
                        system_response_p2 = "what price?"
                    elif not 'food' in preferences:
                        system_response_p2 = "what food type?"
                    system_response = system_response_p1 + system_response_p2 
            else:
                ## meer duidelijkheid hier: vind wat voor flag het is (food area of price) en antwoord gebaseerd daarop.
                if not preferences:
                    if 'food' in user_words:
                        position = user_words.index('food')
                        system_response = f"I'm sorry, there's no {user_words[position-1]} food in town, any other food you'd like?"
                    else: 
                        system_response = "I'm sorry, I didn't understand your preferences. Could you please clarify?"
                elif not 'area' in preferences:
                    system_response = f"I'm sorry but there's no {user_words[0]} area in town, could you change or clarify?"
                elif not 'price' in preferences:
                    system_response = f"I'm sorry but there's no {user_words[0]} price range known, could you change or clarify?"
                elif not 'food' in preferences:
                    system_response = f"I'm sorry but there's no {user_words[0]} food in town, could you change or clarify?"
                else:
                    system_response = f"If you see this something weird happend, but please let me know your prefrences"
                    
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested


def handle_confirm(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = ''
    if state == 'get_preferences':
        next_state = state
        system_response = "I didn\'t quite catch that. Can you phrase that again differently?"
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested

def handle_ack(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = state
        system_response = "Okay. Would you please tell me your preferences?"
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested

def handle_affirm(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = state
        system_response = "Okay. Would you please tell me your preferences?"
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested

def handle_bye(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = state
        system_response = "Okay. Would you please tell me your preferences?"
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested

def handle_deny(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = state
        system_response = "Okay. You were not happy with what I said? Say 'restart' if you want to do over."
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested

def handle_hello(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Hello again! Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = state
        system_response = "Hello again! Would you please tell me your preferences?"
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested
  
def handle_negate(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = state
        system_response = "Okay. You were not happy with what I said? Say 'restart' if you want to do over."
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested

def handle_null(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = state
        system_response = "I didn't quite catch that, can you repeat what you said differently please?"
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested

def handle_repeat(state, user_input, preferences, suggested, rejected):
    #not a parrot, look above
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = 'start'
        system_response = "Okay let's start over."
        preferences.clear()
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested

def handle_reqalts(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = state
        system_response = "Let's continue with collecting your preferences. Can you please tell me your preferences?"
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = 'Try typing restart and let me know your new prefrences.'
        user_words = user_input.lower().split()
        matched = False
        for word in user_words:
            for key, values in flag_dict.items():
                if word in [v.lower() for v in values]:
                    preferences[key] = word
                    matched = True
        if matched:
            next_state = 'suggest_restaurant'
            restaurants, match = recommend_restaurant(preferences, suggested, rejected)
            suggested = random.choice(restaurants)
            if match:
                system_response = f"Great! A restaurant that matches all your preferences is the {restaurant['name']}."
            else:
                system_response = f"We have found no restaurant that matches the entire description of your preferences. But, a close match is {restaurant['name']}, it is a {restaurant['food']} type restaurant, the price range is {restaurant['price']} and it is {restaurant['area']}."
        else:
            min_distance = float('inf')
            best_match = None
            best_key = None
            for word in user_words:
                for key, values in flag_dict.items():
                    for value in values:
                        distance = Levenshtein.distance(word, value.lower())
                        if distance < min_distance:
                            min_distance = distance
                            best_match = value
                            best_key = key
            if min_distance <= 2 and best_match and best_key:
                preferences[best_key] = best_match.lower()
                matched = True
                next_state = 'suggest_restaurant'
                restaurants, match = recommend_restaurant(preferences, suggested, rejected)
                suggested = random.choice(restuarants)
                if match:
                    system_response = f"Great! A restaurant that matches all your preferences is the {restaurant['name']}."
                else: 
                    system_response = f"We have found no restaurant that matches the entire description of your preferences. But, a close match is {restaurant['name']}, it is a {restaurant['food']} type restaurant, the price range is {restaurant['price']} and it is {restaurant['area']}."
            else:
                ## meer duidelijkheid hier: vind wat voor flag het is (food area of price) en antwoord gebaseerd daarop.
                if not preferences:
                    if 'food' in user_words:
                        position = user_words.index('food')
                        system_response = f"I'm sorry, there's no {user_words[position-1]} food in town, any other food you'd like?"
                    else: 
                        system_response = "I'm sorry, I didn't understand your preferences. Could you please clarify?"
                elif not 'area' in preferences:
                    system_response = f"I'm sorry but there's no {user_words[0]} area in town, could you change or clarify?"
                elif not 'price' in preferences:
                    system_response = f"I'm sorry but there's no {user_words[0]} price range known, could you change or clarify?"
                elif not 'food' in preferences:
                    system_response = f"I'm sorry but there's no {user_words[0]} food in town, could you change or clarify?"
                else:
                    system_response = f"If you see this something weird happend, but please let me know your prefrences"
    return next_state, system_response, suggested

def handle_reqmore(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = state
        system_response = "Let's continue with collecting your preferences. Can you please tell me your preferences?"
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        rejected.append(suggested)
        restaurants, match = recommend_restaurant(preferences, suggested, rejected)
        suggested = random.choice(restaurants)
        if match:
            system_response = "Another restaurant that matches your preferences is {suggested}"
        else:
            system_response = "We have found no other restaurant that matches the entire description of your preferences. But, a close match is {restaurant['name']}, it is a {restaurant['food']} type restaurant, the price range is {restaurant['price']} and it is {restaurant['area']}.  Are you happy with this choice?"
    return next_state, system_response, suggested

def handle_request(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = state
        system_response = "Let's continue with collecting your preferences. Can you please tell me your preferences?"
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = f"The restaurants phone number is {suggested['phone']}, the address is {suggested['addr']} and the postcode {suggested['postcode']}"
    return next_state, system_response, suggested

def handle_restart(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = 'start'
        system_response = "Okay let's start over."
        preferences.clear()
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested

def handle_thankyou(state, user_input, preferences, suggested, rejected):
    if state == 'start':
        next_state = state
        system_response = 'Would you please tell me your preferences?'
    if state == 'get_preferences':
        next_state = state
        system_response = "You are welcome! Let's continue. Would you please tell me your preferences?"
    ## NEXT STAGE
    if state == 'suggest_restaurant':
        next_state = state
        system_response = ""
    return next_state, system_response, suggested


dialog_act_handlers = {
    'ack': handle_ack,
    'affirm': handle_affirm,
    'bye': handle_bye,
    'confirm': handle_confirm,
    'deny': handle_deny,
    'hello': handle_hello,
    'inform': handle_inform,
    'negate': handle_negate,
    'null': handle_null,
    'repeat': handle_repeat,
    'reqalts': handle_reqalts,
    'reqmore': handle_reqmore,
    'request': handle_request,
    'restart': handle_restart,
    'thankyou': handle_thankyou,
}
    
if __name__ == '__main__':
    Interactive()     
        


Type 'exit' or 'quit' to end the conversation at any time. Let's begin.
Hello, this is a restaurant finder system. Please tell us about which direction you want to go (north, west etc.), what price range (cheap, moderate, expensive), and what type of food you would like.
Please type in a simple manner, I am not very smart and very rigorous.
user:
any


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


inform
System: Do you have additional requirements?
user:
romantic


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


inform
System: Great! A restaurant that matches all your preferences is the hk fusion, it is romantic.
user:
exit
Thank you for using the restaurant finder system. Goodbye!
